# Project - Laptops Assistant

A simple inventory tool integrated with Anthropic API

In [0]:
# imports

import os
import json
from dotenv import load_dotenv
import anthropic
import gradio as gr

In [0]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

In [0]:
# Connect to OpenAI, Anthropic and Google; comment out the Claude or Google lines if you're not using them

claude = anthropic.Anthropic()

In [0]:
MODEL = "claude-3-haiku-20240307"

In [0]:
system_message = "You are a helpful assistant for an  Inventory Sales called InvAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [0]:
laptop_items = [
    {
        "model": "Aspire 3 A315-59-570Z OPI Pure Silver", 
        "brand": "Acer",
        "price": "$595.96"
    },
    {
        "model": "Aspire Lite 14 AL14-31P-36BE Pure Silver", 
        "brand": "Acer",
        "price": "$463.52"
    },
    {
        "model": "Raider 18 HX",
        "brand": "MSI",
        "price": "$235.25"
    }
]

In [0]:
def get_laptop_price(model):
    print(f"Tool get_laptop_price called for laptop model {model}")
    laptop_model = model.lower()
    for item in laptop_items:
        if laptop_model in item.get("model").lower():
            return item
    return "Unknown"

In [0]:

# get_laptop_price("Lite 14 AL14-31P-36BE Pure SilveR")

get_laptop_price("Aspire Lite 14")

In [0]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_laptop_price",
    "description": (
        "Returns the laptop's price, brand, and exact model from a given query."
        "Use when the user asks about a laptop's price, e.g.,"
        "'How much is this laptop?' → 'The Acer Aspire Lite 14 AL14-31P-36BE Pure Silver is priced at $463.52.'"
    ),
    "input_schema": {
      "type": "object",
      "properties": {
        "model": {
          "type": "string",
          "description": "The model name of the laptop the customer is asking about."
        }
      },
      "required": ["model"]
    }
}

In [0]:
# And this is included in a list of tools:

tools = [price_function]

In [0]:
def chat(message, history):
    print(history)
    messages = [{"role": "user", "content": message}]

    for history_message in history:
        if history_message["role"] == "user":
            messages.append({"role": "user", "content": history_message["content"]})
        
    response = claude.messages.create(model=MODEL, messages=messages, tools=tools, max_tokens=500)

    if len(response.content) > 1:
        assistant, user, laptop_model = handle_tool_call(response)
        messages.append(assistant)
        messages.append(user)
        response = claude.messages.create(model=MODEL, messages=messages, tools=tools, max_tokens=500)


    return response.content[0].text

In [0]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    # laptop_model = message
    laptop_model = message.content[1].input.get("model")
    laptop_item = get_laptop_price(laptop_model)
    assistant = {
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": message.content[0].text
            },
            {
                "type": "tool_use",
                "id": message.content[1].id,
                "name": message.content[1].name,
                "input": message.content[1].input
            }
        ]
    }
    user = {
        "role": "user",
        "content": [
            {
                "type": "tool_result",
                "tool_use_id": message.content[1].id,
                # "content": laptop_item.get("price")
                "content": json.dumps(laptop_item)
            }
        ]
    }
    

    return assistant, user, laptop_model

In [0]:
gr.ChatInterface(fn=chat, type="messages").launch()